In [ ]:
import arcgis
from arcgis.gis import GIS
from arcgis.apps import storymap
from arcgis.mapping import WebMap
from arcgis.geometry import Geometry, SpatialReference, Point, Polygon
from getpass import getpass
from arcgis.features import FeatureLayer, Feature, FeatureLayerCollection
import pandas as pd
import datetime as dt
import os

In [ ]:
username = " "
# Create a connection to your portal for publishing (enter your ArcGIS Online 
#  password in the textbox that appears, then hit 'Enter' on your keyboard)
gis = GIS("https://www.arcgis.com", username, getpass())

In [ ]:
#Find all the community storymaps in our ArcGIS Online content, and save each to a list
find_storymaps = gis.content.search("title: Community Storymap", item_type="Web Mapping Application", max_items=480, outside_org=False)
find_storymaps

In [ ]:
#Set up a log file to write errors to
today = [dt.date.today()]
#log_path = r"T:\CDO_To_Esri\Storymaps\_Archive\Logfiles\Municipalities\Update Contacts"
#log_file = os.path.join(log_path, "Update Contacts_" + str(today[0]) + str(".txt"))
#out = open(log_file,'w')

#For each storymap in the find_storymaps list...
for f in find_storymaps:
    if f.title != 'Alaska DCRA Community Storymaps' and not f.title.startswith('Communit') and 'Community' in str(f.title):
        csmap = storymap.JournalStoryMap(gis.content.get(str(f.id)))
        sections = csmap.properties["values"]["story"]["sections"]
        community = f.title.replace(" Community Storymap", "").replace(" (Unincorporated Community)", "").replace("'","''")
        print(f.title, f.id)
        if community.startswith("City & Borough of "):
            community = community.replace("City & Borough of ", "")
        if community.startswith("Municipality of "):
            community = community.replace("Municipality of ","")
        print(community)

    ###CONTACTS PANEL###
        panel_contacts = sections[10]
    #Find Municipality Contact Information
        #try:
        if 'Unincorporated' not in str(f.title):
            contacts_lyr = FeatureLayer("http://maps.commerce.alaska.gov/arcgis/rest/services/Contacts/Contacts_Municipality/MapServer/0")
            contacts_lyr
            contact_info = contacts_lyr.query(where="City = '" + str(community.replace("'","''")) + "'",out_fields='EntityName,Address,City,Zip,Phone,Fax,Website,Email')
            #Write municipality info to table
            output = str(contact_info.df)
            if output == 'Empty DataFrame\nColumns: []\nIndex: []':
                embed_muni_list = 'There is no available municipality contact information for this community.'
            else: 
                muni_contact = []
                address = contact_info.df.Address[0]
                city = contact_info.df.City[0]
                zipcode = contact_info.df.Zip[0]
                phone = contact_info.df.Phone[0]
                fax = contact_info.df.Fax[0]
                website = contact_info.df.Website[0]
                email = contact_info.df.Email[0]
                muni_contact.append("<p style='text-align:left'><u><b>Municipality Contact Information:</b></u></p>")
                muni_contact.append("<p style='text-align:center'><b>" + str(contact_info.df.EntityName[0]) + '</b></p>')
                muni_contact.append("<p style='text-align:center'>" + str(address) + ", " + str(city) + ", AK" + " " + str(zipcode) + '<p>&nbsp;</p>\n\n')
                muni_contact.append("<p style='text-align:center'>Phone Number: " + str(phone) + '</p>')
                muni_contact.append("<p style='text-align:center'>Fax Number: " + str(fax) + '</p><p>&nbsp;</p>\n\n')
                muni_contact.append("<p style='text-align:center'>Website: " + str(website) + '</p>')
                muni_contact.append("<p style='text-align:center'>" + str(email) + '</p><p>&nbsp;</p>\n\n')
                embed_muni_list = ' '.join(muni_contact)
        else:
            embed_muni_list = ''

        #Reset community queries for boroughs
        if community.endswith(" Borough"):
            community2=community.replace(" Borough", "")
        else:
            community2 = community

        #Federally recognized tribe contact info
        tribe_lyr = FeatureLayer("http://maps.commerce.alaska.gov/arcgis/rest/services/Contacts/CDO_Contacts_Federally_Recognized_Tribes/MapServer/0")
        tribe_query = tribe_lyr.query(where="Community = '" + str(community.replace("'","''")) + "' OR Community = '" + str(community2.replace("'","''")) + "'", out_fields='*')
        if str(tribe_query.df) ==  'Empty DataFrame\nColumns: []\nIndex: []':
            embed_tribe = str(community) + " is not currently served by a federally recognized tribe."
        else:
            tribe_contact = []
            tribes_lst = tribe_query.df.EntityName
            for t in tribes_lst:
                t_query = tribe_lyr.query(where="EntityName = '" + str(t.replace("'","''")) + "'", out_fields = "*")
                tribe = str(t)
                if str(t_query.df) != 'Empty DataFrame\nColumns: []\nIndex: []':
                    tribe_address = t_query.df.Address[0]
                    tribe_city = t_query.df.City[0]
                    tribe_zip = t_query.df.Zip[0]
                    tribe_phone = t_query.df.Phone[0]
                    tribe_fax = t_query.df.Fax[0]
                    tribe_web = t_query.df.Website[0]
                    tribe_email = t_query.df.Email[0]
                    tribe_contact.append("<p style='text-align:center'><b>" + str(tribe) + '</b></p>')
                    tribe_contact.append("<p style='text-align:center'>" + str(tribe_address) + ", " + str(tribe_city) + ", AK" + " " + str(tribe_zip) + '<p>&nbsp;</p>\n\n')
                    tribe_contact.append("<p style='text-align:center'>Phone Number: " + str(tribe_phone) + '</p>')
                    tribe_contact.append("<p style='text-align:center'>Fax Number: " + str(tribe_fax) + '</p><p>&nbsp;</p>\n\n')
                    tribe_contact.append("<p style='text-align:center'>Website: " + str(tribe_web) + '</p>')
                    tribe_contact.append("<p style='text-align:center'>" + str(tribe_email) + '</p><p>&nbsp;</p>\n\n<p>&nbsp;</p>\n\n')
                else:
                    tribe_contact.append("<p style='text-align:center'><b>" + str(tribe) + '</b></p>')
                    tribe_contact.append('<p><i>Contact information for this federally recognized tribe is currently unavailable.</i></p>')
                tribe_info = ' '.join(tribe_contact)

            embed_tribe = tribe_info

        #Native village corporation contact info
        corp_lyr = FeatureLayer("http://maps.commerce.alaska.gov/arcgis/rest/services/Contacts/Contacts_Native_Village_Corporations/MapServer/0")
        corp_query = corp_lyr.query(where="CommunityName = '" + str(community.replace("'","''")) + "' OR CommunityName = '" + str(community2.replace("'","''")) + "'", out_fields = '*')
        corp_output = corp_query.df
        if str(corp_output) == 'Empty DataFrame\nColumns: []\nIndex: []':
            embed_nat_corp = 'There is no native village corporation contact information for this community.'
        else:
            corp_contact = []
            corps = corp_output.EntityName[0]
            corp_address = corp_output.Address[0]
            corp_city = corp_output.City[0]
            corp_zip = corp_output.Zip[0]
            corp_phone = corp_output.Phone[0]
            corp_fax = corp_output.Fax[0]
            corp_web = corp_output.Website[0]
            corp_email = corp_output.Email[0]
            corp_contact.append("<p style='text-align:center'><b>" + str(corps) + '</b></p>')
            corp_contact.append("<p style='text-align:center'>" + str(corp_address) + ", " + str(corp_city) + ", AK" + " " + str(corp_zip) + '<p>&nbsp;</p>\n\n')
            corp_contact.append("<p style='text-align:center'>Phone Number: " + str(corp_phone) + '</p>')
            corp_contact.append("<p style='text-align:center'>Fax Number: " + str(corp_fax) + '</p><p>&nbsp;</p>\n\n')
            corp_contact.append("<p style='text-align:center'>Website: " + str(corp_web) + '</p>')
            corp_contact.append("<p style='text-align:center'>Email: " + str(corp_email) + '</p><p>&nbsp;</p>\n\n')
            embed_nat_corp = ' '.join(corp_contact)
        
        date = str(today[0])
        month = date[5:7]
        day = date[8:10]
        year = date[0:4]
        
        panel_contacts["content"] = '<style type="text/css">.dcra-blue {\n     display: inline-block;\n     background-color: #005e95;\n     border-color: #005e95 !important;\n     color: #fff !important;\n     padding: 0px 4px;\n     border-radius: 0px;\n}\n</style>\n<p><a class="dcra-blue" data-storymaps="MJ-ACTION-1524783785796" data-storymaps-type="navigate">Back to Table of Contents</a></p>\n\n<p>&nbsp;</p>\n\n<div>'+ str(embed_muni_list) + '</div>\n\n<p>&nbsp;</p>\n\n<div><p style="text-align:left"><u><b>Federally Recognized Tribe Contact Information:</b></u></p>' + str(embed_tribe) + '</div><p>\n\n<p>&nbsp;</p>\n\n<p style="text-align:left"><u><b>Native Village Corporation Contact Information:</b></u></p><div>' + str(embed_nat_corp) + '</div><p>\n\n<p>&nbsp;</p><p><i>Contact information last updated on ' + str(month) + '/' + str(day) + '/' + str(year) + '. </i> <p>&nbsp;</p><p>To find all community contacts and to download mailing labels visit the <a href="http://dcced.maps.arcgis.com/apps/webappviewer/index.html?id=148ae1328f9549acb46cfa80366ed37e" target="_blank">Contacts Directory</a>.&nbsp;</p>\n'

        csmap.save()
        print("Contacts data posted to panel.")   

        